# Workflow CAVE + HpBandster

We will present you a example workflow of how to efficiently optimize a algorithm using our frameworks 
<a href="https://github.com/automl/CAVE" target="_blank">CAVE</a> and <a href="https://github.com/automl/HpBandSter" target="_blank">HpBandSter</a>

## Short introduction to the used frameworks
### CAVE

Hier sollte eine kurze Beschreibung von Cave stehen.

### HpBandSter

Modern deep learning methods are very sensitive to many hyperparameters, and, due to the long training times of state-of-the-art models, vanilla Bayesian hyperparameter optimization is typically computationally infeasible. On the other hand, bandit-based configuration evaluation approaches based on random search lack guidance and do not converge to the best configurations as quickly. With HpBanster, we propose to combine the benefits of both Bayesian optimization and bandit-based methods, in order to achieve the best of both worlds: strong anytime performance and fast convergence to optimal configurations. We propose a new practical state-of-the-art hyperparameter optimization method, which consistently outperforms both Bayesian optimization and Hyperband on a wide range of problem types, including high-dimensional toy functions, support vector machines, feed-forward neural networks, Bayesian neural networks, deep reinforcement learning, and convolutional neural networks. Our method is robust and versatile, while at the same time being conceptually simple and easy to implement.

For more insights, please consider the paper: <a href="https://arxiv.org/abs/1807.01774" target="_blank">BOHB: Robust and Efficient Hyperparameter Optimization at Scale</a>

## The Workflow


In the next section, we guide through the following steps:
#### 1) Given a algorithm to optimize and a *configuration space*, we will run BOHB on this problem. 
This will return us the optimization run results. For example the best hyperparameter configuration, which is often referred to as *incumbent*. 
Also a lot of information like which configurations has been used, as well as their performances.
#### 2) We will pass the BOHB results into the CAVE-tool.
It will give insights into the 
+ Parameter importance, 
+ performance analysis,
+ feature analysis and 
+ configuration behaviour.